# vLLM 显存可视化

Visualize vLLM Memory with PyTorch

介绍：使用Torch Snapshot对vLLM运行时的内存状态进行采集，并将数据导入Web端进行可视化展示。通过该方法，能够清晰、直观地掌握vLLM内部主要的显存分配与使用情况。

**本示例要在GPU/NPU机器上完成**


相关文章：[vLLM显存管理详解](https://zhuanlan.zhihu.com/p/1916529253169734444)

Author: kaiyuan

Email: kyxie@zju.edu.cn

## 1 数据采集


pickle数据，打开的web地址：[Link](https://docs.pytorch.org/memory_viz)

**注意：** 如果vLLM版本较高，主进程无法捕捉数据，需要开启：

os.environ["VLLM_ENABLE_V1_MULTIPROCESSING"] = "0"





### 1.1 V100机器示例

测试机器信息：
- Tesla V100-SXM2-32GB
- NVIDIA-SMI 545.23.08
- Driver Version: 545.23.08
- CUDA Version: 12.3

镜像信息：nvcr.io/nvidia/pytorch:25.04-py3

注：该镜像未带vLLM版本

git clone https://github.com/vllm-project/vllm.git

git checkout v0.9.0

模型： Qwen2.5-7B-Instruct

代码：

In [ ]:
# coding=UTF-8
import torch
from vllm import LLM, SamplingParams


if __name__ == "__main__":
    torch.cuda.memory._record_memory_history(max_entries=100000)
    model_name = "/home/kaiyuan/models/Qwen2.5-7B-Instruct"  # 替换为已下载的模型地址
    llm = LLM(model=model_name, dtype='float16')
    n = 16
    # 准备输入提示
    prompts = [
    "Hello, I'm kaiyuan",
    "Do you subscribe InfraTech?",
    ]

    # 设置采样参数
    sampling_params = SamplingParams(
        temperature=0.8,  # 控制生成文本的随机性，值越高越随机
        top_p=0.95,       # 控制采样范围，值越高生成文本越多样化
        max_tokens=50,     # 生成的最大 token 数量
        n=n
    )
    outputs = llm.generate(prompts, sampling_params)
    torch.cuda.memory._dump_snapshot("vllm_snapshot.pickle")
    torch.cuda.memory._record_memory_history(enabled=None)


### 1.2 NPU 910B 机器示例

测试机器信息：
- 910B1
- npu-smi 23.0.6
- Version: 23.0.6

HDK版本：cat /usr/local/Ascend/driver/version.info

```
Version=23.0.6
ascendhal_version=7.35.19
aicpu_version=1.0
tdt_version=1.0
log_version=1.0
prof_version=2.0
dvppkernels_version=1.1
tsfw_version=1.0
Innerversion=V100R001C15SPC009B220
compatible_version=[V100R001C30],[V100R001C13],[V100R001C15]
compatible_version_fw=[7.0.0,7.1.99]
package_version=23.0.6
```

镜像信息：swr.cn-southwest-2.myhuaweicloud.com/ei_ascendcloud_devops/vllm-ascend:v0.8.5rc1-openeuler


模型： Qwen2.5-7B-Instruct

代码：

In [ ]:
# coding=UTF-8
import torch_npu
from torch_npu.contrib import transfer_to_npu
from vllm import LLM, SamplingParams


if __name__ == "__main__":
    torch_npu.npu.memory._record_memory_history(max_entries=100000)
    model_name = "/home/kaiyuan/models/Qwen2.5-7B-Instruct"  # 替换为已下载的模型地址
    llm = LLM(model=model_name, dtype='float16')
    n = 16
    # 准备输入提示
    prompts = [
    "Hello, I'm kaiyuan",
    "Do you subscribe InfraTech?",
    ]

    # 设置采样参数
    sampling_params = SamplingParams(
        temperature=0.8,  # 控制生成文本的随机性，值越高越随机
        top_p=0.95,       # 控制采样范围，值越高生成文本越多样化
        max_tokens=50,     # 生成的最大 token 数量
        n=n
    )
    outputs = llm.generate(prompts, sampling_params)
    torch_npu.npu.memory._dump_snapshot("vllm_snapshot.pickle")
    torch_npu.npu.memory._record_memory_history(enabled=None)


## 2 导入web端可视化

运行结束将生成的vllm_snapshot.pickle文件导入到[https://docs.pytorch.org/memory_viz](https://docs.pytorch.org/memory_viz)。